# Camassa-Holm equation

In [ ]:
include("setup.jl")

# Discretization

In [ ]:
function relaxation_functional(γ, unew, uold, param)
    @unpack D, D2a, tmp1, tmp2 = param
    
    @. tmp2 = (1-γ)*uold + γ*unew
    mul!(tmp1, D2a, tmp2)
    @. tmp1 = tmp2^2 - tmp2 * tmp1
    energy = integrate(tmp1, D)
end
function relaxation_functional(u, param)
    @unpack D, D2a, tmp1 = param
    
    mul!(tmp1, D2a, u)
    @. tmp1 = u^2 - u * tmp1
    energy = integrate(tmp1, D)
end


function save_func_ch_periodic(u, t, integrator)
    @unpack D, D2a, tmp1, usol = integrator.p
    print(".")
    
    mass = integrate(u, D)
    
    mul!(tmp1, D2a, u)
    @. tmp1 = u^2 - u * tmp1
    quadratic = integrate(tmp1, D)
    
    x = grid(D)
    tmp1 .= ( usol.(t, x, x[1], -x[1]) .- u ).^2
    error_u = integrate(tmp1, D) |> sqrt
    
    SVector(mass, quadratic, error_u)
end


function ch_periodic!(du, u, param, t)
    @unpack D, invImD2a, D2b, tmp1, tmp2 = param
    
    # conservative semidiscretization
    mul!(du, D, u)
    @. tmp1 = -u * du
    mul!(tmp2, D2b, tmp1)
    @. tmp1 = tmp1 - 0.5 * tmp2
    mul!(du, D2b, u)
    @. tmp2 = 0.5 * u * du - u^2
    mul!(du, D, tmp2)
    @. tmp1 = tmp1 + du
    ldiv!(du, invImD2a, tmp1)
    
    nothing
end

function solve_ode_ch_periodic(usol, D, D2a, D2b, tspan, alg, tol, dt, adaptive)
    invImD2a = isa(D2a, AbstractMatrix) ? factorize(I - D2a) : I - D2a
    
    x = grid(D)
    u0 = usol.(tspan[1], x, x[1], -x[1])
    tmp1 = similar(u0); tmp2 = similar(tmp1)
    param = (D=D, D2a=D2a, D2b=D2b, invImD2a=invImD2a, tmp1=tmp1, tmp2=tmp2, usol=usol)

    ode = ODEProblem(ch_periodic!, u0, tspan, param)
    
    saveat = range(tspan..., length=100)
    saved_values_baseline = SavedValues(eltype(D), SVector{3,eltype(D)})
    saving_baseline = SavingCallback(save_func_ch_periodic, saved_values_baseline, saveat=saveat)
    saved_values_relaxation = SavedValues(eltype(D), SVector{3,eltype(D)})
    saving_relaxation = SavingCallback(save_func_ch_periodic, saved_values_relaxation, saveat=saveat)
    relaxation = DiscreteCallback((u,t,integrator) -> true, relaxation!, save_positions=(false,false))
    cb_baseline = CallbackSet(saving_baseline)
    cb_relaxation = CallbackSet(relaxation, saving_relaxation)
    
    sol_relaxation = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_relaxation, tstops=saveat, saveat=saveat, maxiters=10^8)
    flush(stdout)
    sol_baseline = solve(ode, alg, abstol=tol, reltol=tol, dt=dt, adaptive=adaptive, save_everystep=false,
        callback=cb_baseline, tstops=saveat, saveat=saveat, maxiters=10^8)
    flush(stdout)

    unum_baseline   = sol_baseline[end]
    unum_relaxation = sol_relaxation[end]
    uana = usol.(tspan[end], x, x[1], -x[1])
    println()
    @printf("Error in u (baseline):   %.3e\n", integrate(u->u^2, unum_baseline - uana, D) |> sqrt)
    @printf("Error in u (relaxation): %.3e\n", integrate(u->u^2, unum_relaxation - uana, D) |> sqrt)
    @printf("Difference of baseline and relaxation in u: %.3e\n", 
        integrate(u->u^2, unum_baseline - unum_relaxation, D) |> sqrt)

    sleep(0.1)
    fig_u, ax = plt.subplots(1, 1)
    plt.plot(x, u0, label=L"u^0")
    plt.plot(x, uana, label=L"$u^\mathrm{ana}$")
    plt.plot(x, unum_baseline, label=L"$u^\mathrm{num}$ (non-conservative)")
    plt.plot(x, unum_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
    plt.xlabel(L"x"); plt.ylabel(L"u")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

    t_baseline = saved_values_baseline.t
    t_relaxation = saved_values_relaxation.t
    mass_baseline      = map(x->x[1], saved_values_baseline.saveval)
    mass_relaxation    = map(x->x[1], saved_values_relaxation.saveval)
    quadratic_baseline   = map(x->x[2], saved_values_baseline.saveval)
    quadratic_relaxation = map(x->x[2], saved_values_relaxation.saveval)

    fig_invariants, ax = plt.subplots(1, 1)
    ax.set_yscale("symlog", linthreshy=1.0e-14)
    plt.plot(t_baseline,   mass_baseline   .- mass_baseline[1], 
        label=L"$\int u$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, mass_relaxation .- mass_relaxation[1], 
        label=L"$\int u$ (conservative)", color="#56B4E9", linestyle="-")
    plt.plot(t_baseline,   quadratic_baseline   .- quadratic_baseline[1], 
        label=L"$\int (u^2 + (\partial_x u)^2)$ (non-conservative)", color="#E69F00", linestyle="--")
    plt.plot(t_relaxation, quadratic_relaxation .- quadratic_relaxation[1], 
        label=L"$\int (u^2 + (\partial_x u)^2)$ (conservative)", color="#56B4E9", linestyle="--")
    plt.xlabel(L"t"); plt.ylabel("Change of Invariants")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
    
    error_u_baseline   = map(x->x[3], saved_values_baseline.saveval)
    error_u_relaxation = map(x->x[3], saved_values_relaxation.saveval)

    fig_error, ax = plt.subplots(1, 1)
    ax.set_xscale("log")
    ax.set_yscale("log")
    plt.plot(t_baseline,   error_u_baseline, 
        label=L"Error of $u$ (non-conservative)", color="#E69F00", linestyle="-")
    plt.plot(t_relaxation, error_u_relaxation, 
        label=L"Error of $u$ (conservative)", color="#56B4E9", linestyle="-")
    plt.xlabel(L"t"); plt.ylabel("Error")
    plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))
        
    (; sol_relaxation, sol_baseline, saved_values_baseline, saved_values_relaxation, 
        fig_u, fig_invariants, fig_error)
end


# Error growth of smooth solitary wave solutions

In [ ]:
# traveling wave solution obtained numerically
c, data = open("ch_traveling_wave_init_c12_ah01_l40_N65536.txt", "r") do io
    line = readline(io)
    line = readline(io)
    line = readline(io)
    c = parse(Float64, line[8:end])
    data = readdlm(io, comments=true)
    c, data
end
x = range(data[1,1], data[end,1], length=size(data,1))
xmin = x[1]; xmax = x[end]
u0 = data[:, 2]
u0itp = CubicSplineInterpolation((x,), u0, extrapolation_bc=Periodic())
get_c() = c
function usol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    u0itp(x_t)
end

In [ ]:
println("c = ", get_c())
println("xmin = ", xmin)
println("xmax = ", xmax)
@show usol(0., xmin, xmin, xmax)
@show usol(0., xmax, xmin, xmax)
@show N = 2^9
@show dt = 0.5 * (xmax - xmin) / (N * get_c())

@show tspan = (0.0, (xmax-xmin)/(3*get_c()) + 200*(xmax-xmin)/get_c())
flush(stdout)

tol = 1.0e-7
adaptive = true
D = fourier_derivative_operator(xmin, xmax, N)
D2a = D^2; D2b = D^2

results = solve_ode_ch_periodic(usol, D, D2a, D2b, tspan, Tsit5(), tol, dt, adaptive);

In [ ]:
ax = results.fig_error.axes[1]
t = [1.0e3, 1.0e4]
ax.plot(t, 4.0e-8 .* t.^2, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{2})", (1.0e3, 2.0e-1), color="gray")
ax.plot(t, 4.0e-8 .* t.^1, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{1})", (1.0e3, 1.5e-4), color="gray")

results.fig_error.savefig(joinpath(dirname(@__DIR__), "figures", "ch_error.pdf"), 
    bbox_inches="tight")

# Error growth of a peakon solution

In [ ]:
xmin = -35.0
xmax = -xmin
get_c() = 1.2
function usol(t, x, xmin, xmax)
    c = get_c()
    x_t = mod(x - c*t - xmin, xmax - xmin) + xmin
    c * exp(-abs(x_t))
end

In [ ]:
println("c = ", get_c())
println("xmin = ", xmin)
println("xmax = ", xmax)
@show usol(0., xmin, xmin, xmax)
@show usol(0., xmax, xmin, xmax)
@show N = 2^13
@show dt = 0.5 * (xmax - xmin) / (N * get_c())

@show tspan = (0.0, (xmax-xmin)/(3*get_c()) + 4*(xmax-xmin)/get_c())
flush(stdout)

tol = 1.0e-7
adaptive = true
D = periodic_derivative_operator(1, 8, xmin, xmax, N+1)
D2a = D2b = periodic_derivative_operator(2, 8, xmin, xmax, N+1)

results = solve_ode_ch_periodic(usol, D, D2a, D2b, tspan, Tsit5(), tol, dt, adaptive);

In [ ]:
t_baseline   = results.saved_values_baseline.t
t_relaxation = results.saved_values_relaxation.t
error_u_baseline   = map(x->x[3], results.saved_values_baseline.saveval)
error_u_relaxation = map(x->x[3], results.saved_values_relaxation.saveval)

fig_error, ax = plt.subplots(1, 1)
ax.set_xscale("log")
ax.set_yscale("log")
plt.plot(t_baseline,   error_u_baseline, 
    label=L"Error of $u$ (non-conservative)", color="#E69F00", linestyle="-")
plt.plot(t_relaxation, error_u_relaxation, 
    label=L"Error of $u$ (conservative)", color="#56B4E9", linestyle="--")
plt.xlabel(L"t"); plt.ylabel(L"Error of $u$")
# plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5))

t = [1.0e1, 1.0e2]
ax.plot(t, 7.0e-5 .* t.^2, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{2})", (2.0e1, 1.0e-1), color="gray")
t = [1.0e1, 2.0e2]
ax.plot(t, 7.0e-4 .* t.^1, ":", color="gray")
ax.annotate(L"\mathcal{O}(t^{1})", (1.0e2, 1.5e-1), color="gray")

fig_error.savefig(joinpath(dirname(@__DIR__), "figures", "ch_peakon_error.pdf"), 
    bbox_inches="tight")

In [ ]:
x = grid(D)
unum_baseline   = results.sol_baseline[end]
unum_relaxation = results.sol_relaxation[end]
uana = usol.(tspan[end], x, x[1], -x[1])

fig_u, ax = plt.subplots(1, 1)
plt.plot(x, unum_baseline, label=L"$u^\mathrm{num}$ (non-conservative)")
plt.plot(x, unum_relaxation, label=L"$u^\mathrm{num}$ (conservative)")
plt.plot(x, uana, label=L"$u^\mathrm{ana}$", ":", color="gray")
plt.xlabel(L"x"); plt.ylabel(L"u")
# plt.legend(loc="center left", bbox_to_anchor=(1.0, 0.5));

fig_u.savefig(joinpath(dirname(@__DIR__), "figures", "ch_peakon_solution.pdf"), 
    bbox_inches="tight")